In [1]:
import mne
import pandas as pd
from tqdm import tqdm

In [2]:
file_path = r'../../Data/ds004196/sub-01/ses-EEG/eeg/sub-01_ses-EEG_task-inner_eeg.bdf'
eeg_data = mne.io.read_raw_bdf(file_path, preload=True)

channels_tsv_path = r'../../Data/ds004196/sub-01/ses-EEG/eeg/sub-01_ses-EEG_task-inner_channels.tsv'
channels_info = pd.read_csv(channels_tsv_path, sep='\t')

events_tsv_path = r'../../Data/ds004196/sub-01/ses-EEG/eeg/sub-01_ses-EEG_task-inner_events.tsv'
events_info = pd.read_csv(events_tsv_path, sep='\t')

montage_path = 'BioSemi64.loc'

Extracting EDF parameters from c:\Users\omarh\Documents\GaTech\Project-InnerSpeech\Data\ds004196\sub-01\ses-EEG\eeg\sub-01_ses-EEG_task-inner_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 696319  =      0.000 ...  1359.998 secs...


In [3]:
def assign_channels(eeg_data, channels_info):
    eeg_data.set_eeg_reference(ref_channels=['Cz'])

    channels_info['name'] = channels_info['name'].replace('T7 (T3)', 'T7')
    channels_info['name'] = channels_info['name'].replace('T8 (T4)', 'T8')
    channels_info['name'] = channels_info['name'].replace('Afz', 'AFz')
    channels_info['name'] = channels_info['name'].replace('Iz (inion)', 'Iz')

    available_channels = set(eeg_data.ch_names).intersection(set(channels_info['name']))
    eeg_data.pick_channels(list(available_channels))
    montage = mne.channels.read_custom_montage(montage_path)
    eeg_data.set_montage(montage, on_missing='ignore')

assign_channels(eeg_data, channels_info)

EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


**Whole Trial**

In [7]:
# Function used to bin data
def bin_eeg_data(eeg_data, events_info):
    events_data = events_info.copy()
    events_data['onset'] = events_info['onset'] / 1000
    events_data['duration'] = events_info['duration'] / 1000

    headers = ['label', 'label_type'] + eeg_data.ch_names
    binned_data = []

    for _, event in tqdm(events_data.iterrows(), total=len(events_data), desc='EEG Event Binning'):
        if event['trial_type'] in ['wife', 'child', 'father', 'daughter']:
            label_type = 'social'
        elif event['trial_type'] in ['four', 'three', 'ten', 'six']:
            label_type = 'numeric'
        else:
            label_type = 'unknown'

        onset = event['onset']
        duration = event['duration']
        label = event['trial_type']
        
        start_sample = int(onset * eeg_data.info['sfreq'])
        stop_sample = int((onset + duration) * eeg_data.info['sfreq'])

        event_dict = {'label': label, 'label_type': label_type}
        for channel in eeg_data.info['ch_names']:
            event_dict[channel] = eeg_data.get_data(picks=channel, start=start_sample, stop=stop_sample)[0].tolist()
        
        binned_data.append(event_dict)

    binned_df = pd.DataFrame(binned_data, columns=headers)
    return binned_df

In [8]:
binned_df = bin_eeg_data(eeg_data, events_info)

EEG Event Binning: 100%|██████████| 319/319 [00:22<00:00, 14.36it/s]


In [9]:
binned_df.head

<bound method NDFrame.head of       label label_type                                                 P2  \
0       six    numeric  [-0.014644316691000265, -0.014645066689614456,...   
1       six    numeric  [-0.01465453542211863, -0.014655785419808947, ...   
2      four    numeric  [-0.014671660390476012, -0.01467153539070698, ...   
3       ten    numeric  [-0.014617847989907737, -0.01461719174112032, ...   
4      four    numeric  [-0.01463266046253803, -0.014630222967041908, ...   
..      ...        ...                                                ...   
314    four    numeric  [-0.012316477242259814, -0.012317414740527555,...   
315    four    numeric  [-0.012489133173235249, -0.012488570674274604,...   
316   three    numeric  [-0.01252700810325194, -0.012525445606139038, ...   
317  father     social  [-0.012614445441689813, -0.012614289191978524,...   
318  father     social  [-0.012611289197521755, -0.0126128204446924, -...   

                                             

In [10]:
binned_df.to_csv('../../Data/Processed/sub01_binned_raw.csv', index=False)

**During Task**

In [4]:
def bin_eeg_data_task(eeg_data, events_info):
    events_data = events_info.copy()
    events_data['onset'] = events_info['onset'] / 1000
    events_data['duration'] = events_info['duration'] / 1000

    headers = ['label', 'label_type'] + eeg_data.ch_names
    binned_data = []

    for _, event in tqdm(events_data.iterrows(), total=len(events_data), desc='EEG Event Binning'):
        if event['trial_type'] in ['wife', 'child', 'father', 'daughter']:
            label_type = 'social'
        elif event['trial_type'] in ['four', 'three', 'ten', 'six']:
            label_type = 'numeric'
        else:
            label_type = 'unknown'

        onset = event['onset']
        duration = event['duration']
        label = event['trial_type']
        
        start_sample = int(onset * eeg_data.info['sfreq'])
        stop_sample = int((onset + duration) * eeg_data.info['sfreq'])

        event_dict = {'label': label, 'label_type': label_type}
        for channel in eeg_data.info['ch_names']:
            event_dict[channel] = eeg_data.get_data(picks=channel, start=start_sample + 512, stop=start_sample + 1536)[0].tolist()
        
        binned_data.append(event_dict)

    binned_df = pd.DataFrame(binned_data, columns=headers)
    return binned_df

In [5]:
binned_df = bin_eeg_data_task(eeg_data, events_info)

EEG Event Binning: 100%|██████████| 319/319 [00:21<00:00, 14.76it/s]


In [6]:
binned_df.to_csv('../../Data/Processed/sub01_binned_task_raw.csv', index=False)